In [48]:
import sympy
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import symfem

In [49]:
x, y, x_1, y_1, x_2, y_2, x_3, y_3, p_1, p_2, p_3 = sympy.symbols("x y x_1 y_1 x_2 y_2 x_3 y_3 p_1 p_2 p_3")
X, Y, u, v = sympy.symbols("X Y u v")
P_1_x, P_2_x, P_1_y, P_2_y = sympy.symbols("P_1_x P_2_x P_1_y P_2_y")

In [50]:
BELL_BASIS = symfem.create_element("triangle", "Bell", 5)
BELL_BASIS_reference = BELL_BASIS.get_basis_functions()

In [51]:
triangle_sym = np.array([
    [x_1, y_1],
    [x_2, y_2],
    [x_3, y_3],
    ])

In [52]:
MAP = BELL_BASIS.reference.get_map_to(triangle_sym)
INV_MAP = BELL_BASIS.reference.get_inverse_map_to(triangle_sym)

In [53]:
U, V = MAP
DX1, DY1 = U.diff(x), U.diff(y)
DX2, DY2 = V.diff(x), V.diff(y)

In [54]:
U_inv, V_inv = INV_MAP

In [55]:
DX_new_1 = (BELL_BASIS_reference[1]*DX1 + BELL_BASIS_reference[2]*DY1)
DX_1 = DX_new_1
DX_new_2 = (BELL_BASIS_reference[7]*DX1 + BELL_BASIS_reference[8]*DY1)
DX_2 = DX_new_2
DX_new_3 = (BELL_BASIS_reference[13]*DX1 + BELL_BASIS_reference[14]*DY1)
DX_3 = DX_new_3

In [56]:
DY_new_1 = (BELL_BASIS_reference[1]*DX2 + BELL_BASIS_reference[2]*DY2)
DY_1 = DY_new_1
DY_new_2 = (BELL_BASIS_reference[7]*DX2 + BELL_BASIS_reference[8]*DY2)
DY_2 = DY_new_2
DY_new_3 = (BELL_BASIS_reference[13]*DX2 + BELL_BASIS_reference[14]*DY2)
DY_3 = DY_new_3

In [57]:
F_1 = BELL_BASIS_reference[0]
F_2 = BELL_BASIS_reference[6]
F_3 = BELL_BASIS_reference[12]

In [58]:
theta = sympy.Matrix([
    [(DX1)**2, (2*DX1*DX2), (DX2)**2],
    [(DX1*DY1), (DY1*DX2 + DX1*DY2), (DX2*DY2)],
    [(DY1)**2, (2*DY1*DY2), (DY2)**2]
])

H = theta.T

In [59]:
DXX_new_1 = (BELL_BASIS_reference[3]*H.row(0)[0] + 
             BELL_BASIS_reference[4]*H.row(0)[1] +
             BELL_BASIS_reference[5]*H.row(0)[2])

DXX_1 = DXX_new_1

In [60]:
DXX_new_2 = (BELL_BASIS_reference[9] *H.row(0)[0] +
             BELL_BASIS_reference[10]*H.row(0)[1] +
             BELL_BASIS_reference[11]*H.row(0)[2])

DXX_2 = DXX_new_2

In [61]:
DXX_new_3 = (BELL_BASIS_reference[15]*H.row(0)[0] +
             BELL_BASIS_reference[16]*H.row(0)[1] +
             BELL_BASIS_reference[17]*H.row(0)[2])
DXX_3 = DXX_new_3

In [62]:
DXY_new_1 = (BELL_BASIS_reference[3]*H.row(1)[0] +
             BELL_BASIS_reference[4]*H.row(1)[1] +
             BELL_BASIS_reference[5]*H.row(1)[2])

DXY_1 = DXY_new_1

In [63]:
DXY_new_2 = (BELL_BASIS_reference[9] *H.row(1)[0] +
             BELL_BASIS_reference[10]*H.row(1)[1] +
             BELL_BASIS_reference[11]*H.row(1)[2])

DXY_2 = DXY_new_2

In [64]:
DXY_new_3 = (BELL_BASIS_reference[15]*H.row(1)[0] +
             BELL_BASIS_reference[16]*H.row(1)[1] +
             BELL_BASIS_reference[17]*H.row(1)[2])

DXY_3 = DXY_new_3

In [65]:
DYY_new_1 = (BELL_BASIS_reference[3]*H.row(2)[0] +
             BELL_BASIS_reference[4]*H.row(2)[1] +
             BELL_BASIS_reference[5]*H.row(2)[2])

DYY_1 = DYY_new_1

In [66]:
DYY_new_2 = (BELL_BASIS_reference[9] *H.row(2)[0] +
             BELL_BASIS_reference[10]*H.row(2)[1] +
             BELL_BASIS_reference[11]*H.row(2)[2])

DYY_2 = DYY_new_2

In [67]:
DYY_new_3 = (BELL_BASIS_reference[15]*H.row(2)[0] +
             BELL_BASIS_reference[16]*H.row(2)[1] +
             BELL_BASIS_reference[17]*H.row(2)[2])

DYY_3 = DYY_new_3

In [72]:
all_basis = [F_1.as_sympy(), F_2.as_sympy(), F_3.as_sympy(), 
            DX_1.as_sympy(), DX_2.as_sympy(), DX_3.as_sympy(), 
            DY_1.as_sympy(), DY_2.as_sympy(), DY_3.as_sympy(), 
            DXX_1.as_sympy(), DXX_2.as_sympy(), DXX_3.as_sympy(),
            DXY_1.as_sympy(), DXY_2.as_sympy(), DXY_3.as_sympy(),
            DYY_1.as_sympy(), DYY_2.as_sympy(), DYY_3.as_sympy()]

In [73]:
result = {}

N = len(all_basis)

for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):

    first = all_basis[idx].copy()
    second = all_basis[jdx].copy()

    u_x = \
    sympy.diff(first, x)*P_1_x + \
    sympy.diff(first, y)*P_2_x
    
    u_xx = \
    sympy.diff(u_x, x)*P_1_x + \
    sympy.diff(u_x, y)*P_2_x

    u_y = \
    sympy.diff(first, x)*P_1_y + \
    sympy.diff(first, y)*P_2_y
    
    u_yy = \
    sympy.diff(u_y, x)*P_1_y + \
    sympy.diff(u_y, y)*P_2_y

    u_xy = \
    sympy.diff(u_x, x)*P_1_y + \
    sympy.diff(u_x, y)*P_2_y

    
    v_x = \
    sympy.diff(second, x)*P_1_x + \
    sympy.diff(second, y)*P_2_x

    v_xx = \
    sympy.diff(v_x, x)*P_1_x + \
    sympy.diff(v_x, y)*P_2_x
    
    v_y = \
    sympy.diff(second, x)*P_1_y + \
    sympy.diff(second, y)*P_2_y

    v_yy = \
    sympy.diff(v_y, x)*P_1_y + \
    sympy.diff(v_y, y)*P_2_y
    
    v_xy = \
    sympy.diff(v_x, x)*P_1_y + \
    sympy.diff(v_x, y)*P_2_y

    
    A = u_xx * v_xx
    B = u_xy * v_xy
    C = u_yy * v_yy
    b = u_xx * v_yy

    sym_int =   sympy.integrate(A + 2*B + C, (y, 0, 1-x), (x, 0, 1))
    sym_int_2 = sympy.integrate(A + 2*b + C, (y, 0, 1-x), (x, 0, 1))

    args = [P_1_x, P_2_x, P_1_y, P_2_y]
    
    result[f"{idx}_{jdx}"] =       sympy.lambdify(args, sym_int)
    result[f"{idx}_{jdx}_alter"] = sympy.lambdify(args, sym_int_2)    

  2%|▉                                        | 7/324 [01:22<1:02:26, 11.82s/it]

KeyboardInterrupt



In [ ]:
for idx, basis in enumerate(all_basis_list):
    basis_norm = basis.copy()
    sym_int = sympy.integrate(basis_norm, (y, 0, 1-x), (x, 0, 1))    
    result[f"{idx}"] = sym_int

In [ ]:
import dill
dill.settings['recurse'] = True
dill.dump(result, open("../calculations/i_think_final", "wb"))

In [ ]:
P_1_x, P_2_x, P_1_y, P_2_y = sympy.symbols("P_1_x P_2_x P_1_y P_2_y")

In [ ]:
for idx, basis in enumerate(all_basis):
    
    func = basis.subs(x, p_1).subs(y, p_2)

    func_x = sympy.diff(func, p_1)*P_1_x + sympy.diff(func, p_2)*P_2_x
    func_y = sympy.diff(func, p_1)*P_1_y + sympy.diff(func, p_2)*P_2_y
    
    func_xx = sympy.diff(func_x, p_1)*P_1_x + sympy.diff(func_x, p_2)*P_2_x + \
              sympy.diff(func_y, p_1)*P_1_x + sympy.diff(func_y, p_2)*P_2_x
    
    func_yy = sympy.diff(func_y, p_1)*P_1_y + sympy.diff(func_y, p_2)*P_2_y + \
              sympy.diff(func_x, p_1)*P_1_y + sympy.diff(func_x, p_2)*P_2_y
    
    func_xy = sympy.diff(func_x, p_1)*P_1_y + sympy.diff(func_x, p_2)*P_2_y + \
              sympy.diff(func_y, p_1)*P_1_x + sympy.diff(func_y, p_2)*P_2_x

    func = func_x
    
    val_1 = func.subs(p_1, 0).subs(p_2, 0)
    val_2 = func.subs(p_1, 1).subs(p_2, 0)
    val_3 = func.subs(p_1, 0).subs(p_2, 1)
    
    print(idx, ":", val_1, val_2, val_3)